First part:

To download SPICE database and generate .ac files for monomers by antechamber

In [ ]:
import h5py
import numpy as np
import os, re
import shutil
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import RDLogger
import openbabel as ob
from openbabel import pybel as pb
from multiprocessing import Pool
RDLogger.DisableLog('rdApp.*') #hiding the warning messages

In [ ]:
os.system('wget -c https://zenodo.org/record/7606550/files/SPICE-1.1.3.hdf5?download=1 -O SPICE-1.1.3.hdf5')
# os.system('mkdir frcmod GAFF_ac2 sdf smiles')
current_path = os.path.abspath('.')
    
nb_cpu = 30
for i in range(nb_cpu):
    os.system('mkdir tmp' + str(i))

In [ ]:
f = h5py.File('SPICE-1.1.3.hdf5','r')
all_key = []
for key in f.keys():
    smiles = f[key]['smiles'][0].decode()
    if 'Li' in smiles or 'Na' in smiles or 'Mg' in smiles or 'K' in smiles or 'Ca' in smiles:
        continue
    if '.' in smiles:
        continue
    mol = pb.readstring('smiles', smiles) 
    mol.make3D() 
    mol.write('smi', './smiles/'+ key +'.txt', overwrite=True)
    mol.write('sdf', './sdf/'+ key +'.sdf', overwrite=True)
    if os.path.exists(current_path + '/GAFF_ac2/' + key + '.ac'):
        continue
    all_key.append(key)

print('Number of monomers:       ', len(all_key))

In [ ]:
# Create the .ac files for all monomers.
# It is possible to clear the current output of notebook! 
def para_process(i):
    os.chdir(current_path + '/tmp' + str(i))
    new_all_key = all_key[i*nb_processing:(i+1)*nb_processing]
    for i, key in enumerate(new_all_key):
        smiles = f[key]['smiles'][0].decode()
        os.system('rm *')
        mol = pb.readstring('smiles', smiles) 
        mol.make3D(forcefield='gaff') 
        mol.write('mdl', './tmp.mdl', overwrite=True)

        nc = 0 # net charge
        for atom in mol.atoms:
            nc += atom.formalcharge
        if not os.system('antechamber -i tmp.mdl -fi mdl -o tmp.ac -fo ac -c bcc -nc ' + str(int(nc)) + ' -pf y -at gaff2'):
            shutil.copyfile('tmp.ac', '../GAFF_ac2/' + key + '.ac')
    return i

nb_processing = int(len(all_key) / nb_cpu) + 1
with Pool(processes=nb_cpu) as pool:
    for results in pool.imap(para_process,range(nb_cpu)):
        pass
os.chdir(current_path)

In [ ]:
# If the molecule cannot give good .ac file by .mdl file, we can only try .pdb file 
def para_process(i):
    os.chdir(current_path + '/tmp' + str(i))
    new_all_key = all_key[i*nb_processing:(i+1)*nb_processing]
    for i, key in enumerate(new_all_key):
        smiles = f[key]['smiles'][0].decode()
        os.system('rm *')
        mol = pb.readstring('smiles', smiles) 
        mol.make3D(forcefield='gaff') 
        mol.write('pdb', './tmp.pdb', overwrite=True)

        nc = 0 # net charge
        for atom in mol.atoms:
            nc += atom.formalcharge
        if not os.system('antechamber -i tmp.pdb -fi pdb -o tmp.ac -fo ac -c bcc -nc ' + str(int(nc)) + ' -pf y -at gaff2'):
            shutil.copyfile('tmp.ac', '../GAFF_ac2/' + key + '.ac')
    return i

new_all_key = []
for key in all_key:
    if not os.path.exists(current_path + '/GAFF_ac2/' + key + '.ac'):
        new_all_key.append(key)
all_key = new_all_key
with Pool(processes=nb_cpu) as pool:
    for results in pool.imap(para_process,range(nb_cpu)):
        pass
os.chdir(current_path)

In [ ]:
# create the complementary GAFF parameter files by parmchk2
def para_process(i):
    os.chdir(current_path + '/tmp' + str(i))
    new_all_name = all_name[i*nb_processing:(i+1)*nb_processing]
    for i, name in enumerate(new_all_name):
        os.system('rm *')
        if not os.system('parmchk2 -i ../GAFF_ac2/' + name + '.ac -f ac -o tmp'):
            os.system('mv tmp ../frcmod/' + name)
    return i

all_name = []
for root, dirs, files in os.walk(current_path + '/GAFF_ac2'):
    for file in files:
        all_name.append(re.split(r'(.ac)',file)[0])
nb_processing = int(len(all_name) / nb_cpu) + 1
with Pool(processes=nb_cpu) as pool:
    for results in pool.imap(para_process,range(nb_cpu)):
        pass
os.chdir(current_path)

Second part:

As the dimers or with plus monomers cannot be processed by antechamber, they have to be treated specifically.

Please restart the kernel!

In [ ]:
import h5py
import numpy as np
import os, re
import shutil
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import RDLogger
import openbabel as ob
from openbabel import pybel as pb
from multiprocessing import Pool
RDLogger.DisableLog('rdApp.*') #hiding the warning messages

current_path = os.path.abspath('.')
nb_cpu = 30


In [ ]:
f = h5py.File('SPICE-1.1.3.hdf5','r')
all_key = []
for key in f.keys():
    smiles = f[key]['smiles'][0].decode()
    if 'Li' in smiles or 'Na' in smiles or 'Mg' in smiles or 'K' in smiles or 'Ca' in smiles :
        continue
    if '.' not in smiles:
        continue
    mol = pb.readstring('smiles', smiles) 
    mol.make3D() 
    mol.write('smi', './smiles/'+ key +'.txt', overwrite=True)
    mol.write('sdf', './sdf/'+ key +'.sdf', overwrite=True)
    if os.path.exists(current_path + '/GAFF_ac2/' + key + '.ac'):
        continue
    all_key.append(key)

print('Number of dimers or plus: ', len(all_key))

In [ ]:
# create the .ac files by antechamber and complementary GAFF parameter files by parmchk2
def para_process(m):
    os.chdir(current_path + '/tmp' + str(m))
    new_all_key = all_key[m*nb_processing:(m+1)*nb_processing]
    for key in new_all_key:
        os.system('rm *')
        bool_ok = True
        smiles = f[key]['smiles'][0].decode()
        part_smi = re.split(r'\.', smiles)
        list_frcmod = []
        for i in range(len(part_smi)):
            mol = pb.readstring('smiles', part_smi[i]) 
            mol.make3D(forcefield='gaff') 
            mol.write('mdl', str(i)+'.mdl', overwrite=True)

            nc = 0
            for atom in mol.atoms:
                nc += atom.formalcharge

            if len(mol.atoms) == 1:
                element = re.findall(r'\[([^\]]+):', part_smi[i])[0]
                if element in ['Br-', 'I-', 'Cl-', 'F-']:
                    shutil.copyfile('../SpecialFile/' + element[:-1] + '.ac', str(i) + '.ac')
                else:
                    bool_ok = False
                    print('wrong: ', element)
                    break
            elif len(mol.atoms) == 2 and 'H' in part_smi[i]:
                shutil.copyfile('../SpecialFile/HH.ac', str(i) + '.ac')
            else:
                if os.system('antechamber -i ' + str(i) + '.mdl -fi mdl -o ' + str(i) + '.ac -fo ac -pf y -c bcc -nc ' + str(nc) + ' -at gaff2'):
                    mol.write('pdb', str(i)+'.pdb', overwrite=True)
                    if os.system('antechamber -i ' + str(i) + '.pdb -fi pdb -o ' + str(i) + '.ac -fo ac -pf y -c bcc -nc ' + str(nc) + ' -at gaff2'):
                        bool_ok = False
                        break

                if os.system('parmchk2 -i '+str(i)+'.ac -f ac -o ' + str(i)):
                    bool_ok = False
                    break
                else:
                    list_frcmod.append(i)
        if bool_ok:
            ac_file = open(current_path + '/GAFF_ac2/' + key + '.ac', 'w+')
            for i in range(len(part_smi)):
                ac_file.write(open(str(i)+'.ac','r').read()+'\n')
            ac_file.close()
            
        if len(list_frcmod) > 0:
            frcmod_file = open(current_path + '/frcmod/' + key, 'w+')
            for i in list_frcmod:
                frcmod_file.write(open(str(i),'r').read()+'\n')
            frcmod_file.close()
        else:
            shutil.copyfile('../SpecialFile/frcmod', '../frcmod/' + key)
    return m

nb_processing = int(len(all_key) / nb_cpu) + 1
with Pool(processes=nb_cpu) as pool:
    for results in pool.imap(para_process,range(nb_cpu)):
        pass
os.chdir(current_path)

In [ ]:
# delete the tmp directory
os.system('rm -rf tmp*')

Third part:

Recored all molecules that can be processed correctly and store them on 'mol.txt'

You should restart the kernel!

In [ ]:
import os
import re
import random
import numpy as np
import functools
from multiprocessing import Pool
import h5py
# calculate the molecules can/cannot be processed

nb_cpu=30
current_path = os.path.abspath('.')
f = h5py.File('SPICE-1.1.3.hdf5','r')
success_key = []
fail_key = []
for key in f.keys():
    if os.path.exists(current_path + '/sdf/' + key + '.sdf') and os.path.exists(current_path + '/GAFF_ac2/' + key + '.ac'):
        success_key.append(key)
    else:
        fail_key.append(key)
print(len(success_key), 'molecules are ok!')
print(len(fail_key), 'molecules are failed!')

os.chdir(current_path + '/../../')
from src.ForceFields.GAFF.ReadPara import read_GAFF2_dat
from src.SPICE.ReadMol import para_process_mol

mol = []
nb_time_processing = 20

GAFF = read_GAFF2_dat(False, False)
with Pool(processes=nb_cpu) as pool:
    for results in pool.imap(functools.partial(para_process_mol, all_name=success_key, dataset='SPICE', GAFF=GAFF, bond_morse=False, angle_ub=False, max_len=100, nb_time_processing=nb_time_processing),range(int(np.ceil(len(success_key)/nb_time_processing)))):
        i, mol_part = results
        mol += mol_part
        if (i+1) % (10*nb_cpu) == 0:
            print(i+1,'/',int(np.ceil(len(all_name)/nb_time_processing)),' finished')
print('Molecule processing finished!')


In [ ]:
random.shuffle(mol) 
f = h5py.File('./data/SPICE/SPICE-1.1.3.hdf5','r')
subset = []
for key in f.keys():
    name = f[key]['subset'][:][0].decode()
    if name not in subset:
        subset.append(name)

aa = np.zeros(len(subset))
bb = []
for _ in range(len(subset)):
    bb.append([])

for i in range(len(mol)):
    name = f[mol[i].mol_name]['subset'][:][0].decode()
    for j in range(len(subset)):
        if subset[j] == name:
            aa[j] += 1
            bb[j].append(mol[i].mol_name)

train = []
val = []
test = []
for i in range(len(subset)):
    num_train = int(0.8 * aa[i])
    num_val = int(0.9 * aa[i])
    train += bb[i][:num_train]
    val += bb[i][num_train:num_val]
    test += bb[i][num_val:]
    
conf = np.zeros(3)
for i in train:
    conf[0] += len(f[i]['conformations'][:])
for i in val:
    conf[1] += len(f[i]['conformations'][:])
for i in test:
    conf[2] += len(f[i]['conformations'][:])
print('Conformation in Train/Val/Test datasets:')
print('Train      ', conf[0])
print('Validation ', conf[1])
print('Test       ', conf[2])
#assert the number of conformation in each dataset is ok
print('If you are not satisfied, please rerun this block!')

xyz_file = open('./data/SPICE/mol.txt', 'w+')
for i in train:
    xyz_file.write(i+'\n')
for i in val:
    xyz_file.write(i+'\n')
for i in test:
    xyz_file.write(i+'\n')
xyz_file.close()
